In [1]:
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np
from scipy.stats import norm
from sklearn.utils import shuffle
from math import sqrt
from sklearn.datasets import fetch_mldata
from sklearn.preprocessing import scale, LabelEncoder
from sklearn.model_selection import GridSearchCV, train_test_split

In [2]:
class SCW1(BaseEstimator, ClassifierMixin):
    def __init__(self, C=1.0, eta=0.90, n_iter=3, random_state=None, shuffle=True):
        self.C = C
        self.eta = eta
        self.n_iter = n_iter
        self.shuffle = shuffle
        self.random_state = random_state
        self.phi = norm.cdf(self.eta)**(-1)
        self.psi = 1 + self.phi**2 / 2
        self.zeta = 1 + self.phi**2
        
    def fit(self, X, Y):
        n_samples, n_features = X.shape
        self.mu = np.zeros(n_features).reshape(n_features, 1)
        self.sigma = np.diag([1.0] * n_features)
        
        for epoch in range(self.n_iter):
            if self.shuffle:
                X, Y = shuffle(X, Y, random_state=self.random_state)
            
            for i in range(n_samples):
                self._update(X[i:i + 1].T, Y[i:i + 1])

    def _update(self, X, y):
        m = float(y * self.mu.T @ X)
        v = float(X.T @ (self.sigma @ X))
        alpha = min(self.C, max(0, (-m * self.psi + sqrt(m**2 * self.phi**4 / 4 + v * self.phi**2 * self.zeta)) / (v * self.zeta)))
        u = (-alpha * v * self.phi + sqrt(alpha**2 * v**2 * self.phi ** 2 + 4 * v))**2 / 4
        beta = alpha * self.phi / (sqrt(u) + v * alpha * self.phi)

        self.mu = self.mu + alpha * y * self.sigma @ X
        self.sigma = self.sigma - beta * self.sigma @ X @ X.T @ self.sigma
        
    def predict(self, X):
        return np.sign(X @ self.mu)

In [ ]:
usps = fetch_mldata('usps')
X = scale(usps.data)
y = usps.target
y_bin = np.array([1 if i >= 5 else -1 for i in y])

X_train, X_test, y_train, y_test = train_test_split(X, y_bin, test_size=0.3, random_state=0)

scw1 = GridSearchCV(
    estimator = SCW1(n_iter=3, random_state=0, shuffle=True),
    param_grid = {'C': [2**-4, 2**-3, 2**-2, 2**-1, 1, 2, 4, 8, 16], 'eta': [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]},
    cv = 3,
    verbose = 1
)
scw1.fit(X_train, y_train)
print('the best C was  : ', scw1.best_params_['C'])
print('the best eta was: ', scw1.best_params_['eta'])
print('usps_SCW1       : {0:.4f}'.format(scw1.score(X_test, y_test)))

In [6]:
svmguide3 = fetch_mldata('svmguide3')
X = scale(svmguide3.data)
y = svmguide3.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

scw1 = GridSearchCV(
    estimator = SCW1(n_iter=5, random_state=None, shuffle=True),
    param_grid = {'C': [2**-4, 2**-3, 2**-2, 2**-1, 1, 2, 4, 8, 16], 'eta': [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]},
    cv = 3,
    verbose = 1
)
scw1.fit(X_train, y_train)
print('the best C was  : ', scw1.best_params_['C'])
print('the best eta was: ', scw1.best_params_['eta'])
print('svmguide3_SCW1  : {0:.4f}'.format(scw1.score(X_test, y_test)))

Fitting 3 folds for each of 90 candidates, totalling 270 fits
the best C was  :  0.5
the best eta was:  0.6
svmguide3_SCW1  : 0.7513


[Parallel(n_jobs=1)]: Done 270 out of 270 | elapsed:   19.8s finished
